# Quickstart

In this quickstart you will create a evaluate a LangChain app using ground truth.

### Add API keys
For this quickstart, you will need Open AI keys.

In [1]:
import os
os.environ["OPENAI_API_KEY"] = "..."
os.environ["HUGGINGFACE_API_KEY"] = "..."

### Import from LlamaIndex and TruLens

In [2]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts.chat import ChatPromptTemplate, PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate

In [3]:
# Imports main tools:
from trulens_eval import Feedback, Tru, TruChain
from trulens_eval.feedback import GroundTruthAgreement, Huggingface
tru = Tru()

### Create Simple LLM Application

This example uses Langchain with an OpenAI LLM.

In [4]:
full_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=
        "Provide an answer to the following: {prompt}",
        input_variables=["prompt"],
    )
)

chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

llm = OpenAI(temperature=0.9, max_tokens=128)

chain = LLMChain(llm=llm, prompt=chat_prompt_template, verbose=True)

## Initialize Feedback Function(s)

In [5]:
golden_set = [
    {"query": "who invented the lightbulb?", "response": "Thomas Edison"},
    {"query": "¿quien invento la bombilla?", "response": "Thomas Edison"}
]

def golden_set_lookup(prompt):
    responses = [qr["response"] for qr in golden_set if qr["query"] == prompt]
    if responses:
        return responses[0]
    else:
        return None


f_groundtruth = Feedback(GroundTruthAgreement(golden_set_lookup).agreement_measure).on_input_output()

# Define a language match feedback function using HuggingFace.
hugs = Huggingface()
f_lang_match = Feedback(hugs.language_match).on_input_output()

✅ In agreement_measure, input prompt will be set to *.__record__.main_input or `Select.RecordInput` .
✅ In agreement_measure, input response will be set to *.__record__.main_output or `Select.RecordOutput` .
✅ In language_match, input text1 will be set to *.__record__.main_input or `Select.RecordInput` .
✅ In language_match, input text2 will be set to *.__record__.main_output or `Select.RecordOutput` .


## Instrument chain for logging with TruLens

In [6]:
tc = TruChain(chain, feedbacks=[f_groundtruth, f_lang_match])

✅ app app_hash_1a2a7ecba192085a90df9c840425270f -> default.sqlite
✅ feedback def. feedback_definition_hash_4bc7b9a91bb270fc24d2ea46ce4545d3 -> default.sqlite
✅ feedback def. feedback_definition_hash_feaf623d759c8edde30241bf7c3adccb -> default.sqlite


In [7]:
# Instrumented query engine can operate like the original:
tc("¿quien invento la bombilla?")
tc("who invented the lightbulb?")



> Entering new LLMChain chain...

> Finished chain.


> Entering new LLMChain chain...
✅ record record_hash_a87bb46439c39b3fee3a03992ae96b81 from app_hash_1a2a7ecba192085a90df9c840425270f -> default.sqlite
✅ feedback feedback_result_hash_ea6d51c93b710afb5062fd2273ee9a5c on record_hash_a87bb46439c39b3fee3a03992ae96b81 -> default.sqlite
✅ feedback feedback_result_hash_6eff6c356ee64c0f38d890826d22357b on record_hash_a87bb46439c39b3fee3a03992ae96b81 -> default.sqlite

> Finished chain.


{'prompt': 'who invented the lightbulb?',
 'text': '\n\nThomas Edison is credited with inventing the lightbulb.'}

## Explore in a Dashboard

In [8]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

✅ record record_hash_fd28b4dc3ecfa2d7a7faabf0e5db78cf from app_hash_1a2a7ecba192085a90df9c840425270f -> default.sqlite
Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.4.23:8502 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>